In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('all_invoice.*txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [3]:
text

'id\ttext\ttag\n031.jpg\tDate:\tB-DATE\n031.jpg\t01/02/2020\tI-DATE\n031.jpg\tBILL\tO\n031.jpg\tOF\tO\n031.jpg\tLADING\tO\n031.jpg\tPage\tO\n031.jpg\t1\tO\n031.jpg\tSHIP\tO\n031.jpg\tFROM\tO\n031.jpg\tFLOORS\tO\n031.jpg\tLLC\tO\n031.jpg\tBill\tB-INVOICE_NO\n031.jpg\tof\tI-INVOICE_NO\n031.jpg\tlading\tI-INVOICE_NO\n031.jpg\tnumber:\tI-INVOICE_NO\n031.jpg\t7357885\tI-INVOICE_NO\n031.jpg\tpautow,\tO\n031.jpg\t"ca\tO\n031.jpg\t30720\tO\n031.jpg\t1.00E+16\tO\n031.jpg\tSHIP\tO\n031.jpg\tTO\tO\n031.jpg\t30418\tO\n031.jpg\tName..:\tO\n031.jpg\tPCD\tO\n031.jpg\tPACIFIC\tO\n031.jpg\tCOAST\tO\n031.jpg\tDIST\tO\n031.jpg\tG\tB-CUST_NAME\n031.jpg\tW\tI-CUST_NAME\n031.jpg\tPAULSON\tI-CUST_NAME\n031.jpg\tCO\tI-CUST_NAME\n031.jpg\tR\tO\n031.jpg\tNA\tO\n031.jpg\t10855\tO\n031.jpg\tSW\tO\n031.jpg\tCASCADE\tO\n031.jpg\tAVE\tO\n031.jpg\tTIGARD\tO\n031.jpg\tTrailer\tO\n031.jpg\tNumber\tO\n031.jpg\tBILL\tO\n031.jpg\tFREIGHT\tO\n031.jpg\tTO:\tO\n031.jpg\tENGINEERED\tB-COMPANY_NAME\n031.jpg\tFLOORS,\tI-COMPANY

In [4]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [5]:
df = pd.DataFrame(data[1:],columns=data[0])

In [6]:
df.head()

,id,text,tag
0,031.jpg,Date:,B-DATE
1,031.jpg,01/02/2020,I-DATE
2,031.jpg,BILL,O
3,031.jpg,OF,O
4,031.jpg,LADING,O


In [7]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [8]:
df['text'] = df['text'].apply(cleanText)

In [9]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

/tmp/ipykernel_29169/3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [10]:
dataClean.head(10)

,id,text,tag
0,031.jpg,date,B-DATE
1,031.jpg,01/02/2020,I-DATE
2,031.jpg,bill,O
3,031.jpg,of,O
4,031.jpg,lading,O
5,031.jpg,page,O
6,031.jpg,1,O
7,031.jpg,ship,O
8,031.jpg,from,O
9,031.jpg,floors,O


In [11]:
group = dataClean.groupby(by='id')

In [12]:
cards = group.groups.keys()

In [13]:
allCardsData = []
for card in cards:
    cardData = []
    grouparray = group.get_group(card)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    cardData = (content,annotations)
    allCardsData.append(cardData)

In [14]:
allCardsData

[('corporate packaging  promotions bill of lading to consignee — sams dist.ctr 6493 street 1000 s. cucamonga ave   bill of lading number 8175105 po bill date from ar50 corporate packaging  shipper     destination      city/state/zip ontario, ca 91761      third party billing, send to name arizona beverages llc c ompany arizona beverages llc street city/state/zip street 1555 s. archibald avenue origin city/state/zip ontario, ca 91761     special instructions shippers instructions      no. shipping cases no. shipping pallets description of articles  special marks  exceptions   equip 317 7 equip arivat 12/20/19 459 carrier bawb ° seal 3289785 remit c.0.d. address reseal door/zone del date appointment 12/20/19 500  have read and understan appointment / drop rules note w here the rate is depende on value, shippers are required to state specifically in writing the agre or declared value of the property. 7 agreed or declared value of the driver signature property is hereby specifically state 

In [14]:
import random

In [15]:
random.shuffle(allCardsData)

In [16]:
len(allCardsData)

60

In [17]:
TrainData = allCardsData[:40]
TestData = allCardsData[40:]

In [18]:
import pickle

In [19]:
pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))